In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

llm = ChatOpenAI()

memory = ConversationBufferMemory(return_messages=True, return_message=True,)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./rag_data/chapter_3.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

prompt_template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. {context}

Question: {question}
Answer: 
"""

prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
)

chain = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff", 
	retriever=vectorstore.as_retriever(),
	memory=memory,
	chain_type_kwargs={"prompt":prompt},
)

def invoke_chain(question):
	result = chain.invoke({"query":question})
	print(f"Question: {question}")
	print(f"Answer: {result['result']}")

In [20]:
invoke_chain("Aaronson 은 유죄인가요?")

Question: Aaronson 은 유죄인가요?
Answer: I don't know.


In [21]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

Question: 그가 테이블에 어떤 메시지를 썼나요?
Answer: 그가 적은 메시지는 "FREEDOM IS SLAVERY"과 "TWO AND TWO MAKE FIVE"이었습니다.


In [22]:
invoke_chain("Julia 는 누구인가요?")

Question: Julia 는 누구인가요?
Answer: Julia is a character in the novel who is involved in a romantic relationship with Winston, the main character.
